# plfeature Usage Example

This notebook demonstrates how to use the plfeature library for:
1. **PDB Standardization** - Clean and standardize protein structures
2. **Protein Featurization** - Extract structural features from proteins
3. **Molecule Featurization** - Extract features from small molecules (ligands)

We'll use the PDB structure 10gs (Glutathione S-transferase) as an example.

In [ ]:
import sys
import os

# Add parent directory to path to import plfeature
sys.path.insert(0, os.path.dirname(os.getcwd()))

from plfeature import PDBStandardizer, ProteinFeaturizer, MoleculeFeaturizer
import torch

## 1. PDB Standardization

The PDB standardizer cleans and standardizes protein structures:
- Removes hydrogen atoms (optional)
- Removes water molecules and nucleic acids
- Handles post-translational modifications (PTMs)
- Reorders atoms according to standard definitions
- Renumbers residues sequentially

In [ ]:
# Input and output file paths
input_pdb = '10gs_protein.pdb'
output_pdb = '10gs_protein_standardized.pdb'

# Create standardizer
standardizer = PDBStandardizer(
    remove_hydrogens=True,      # Remove hydrogen atoms
    ptm_handling='base_aa'      # Convert PTMs to base amino acids
)

# Standardize the PDB file
standardizer.standardize(input_pdb, output_pdb)
print(f"✓ Standardized PDB saved to: {output_pdb}")

### PTM Handling Modes

The standardizer supports three modes for handling post-translational modifications:

- **`'base_aa'`** (default): Convert PTMs to their base amino acids (e.g., SEP → SER)
  - Use for: ESM embeddings, molecular dynamics simulations
  
- **`'preserve'`**: Keep PTM atoms intact
  - Use for: Protein-ligand modeling where PTM chemistry matters
  
- **`'remove'`**: Remove PTM residues entirely
  - Use for: Standard amino acid-only analysis

## 2. Protein Featurization

Extract structural features from the protein at two levels:
- **Atom-level features**: Individual atom properties and interactions
- **Residue-level features**: Residue properties, SASA, dihedral angles, etc.

### 2.1 Atom-level Features

In [ ]:
# Create atom featurizer
atom_featurizer = ProteinFeaturizer.AtomFeaturizer(output_pdb)

# Get features
atom_features = atom_featurizer.get_features()

print("Atom-level Features:")
print(f"  Number of atoms: {atom_features['node']['node_scalar_features'][0].shape[0]}")
print(f"  Number of edges: {len(atom_features['edge']['edges'][0])}")
print(f"  Scalar feature dimensions: {[f.shape for f in atom_features['node']['node_scalar_features']]}")
print(f"  Vector feature dimensions: {[f.shape for f in atom_features['node']['node_vector_features']]}")

### 2.2 Residue-level Features

In [ ]:
# Create residue featurizer
residue_featurizer = ProteinFeaturizer.ResidueFeaturizer(output_pdb)

# Get amino acid sequence by chain
sequences = residue_featurizer.get_sequence_by_chain()
print("\nProtein Sequences by Chain:")
for chain_id, sequence in sequences.items():
    print(f"  Chain {chain_id}: {sequence[:50]}{'...' if len(sequence) > 50 else ''}")
    print(f"             Length: {len(sequence)} residues")

# Get residue-level features
residue_features = residue_featurizer.get_features()

print("\nResidue-level Features:")
print(f"  Number of residues: {residue_features['node']['node_scalar_features'][0].shape[0]}")
print(f"  Number of edges: {len(residue_features['edge']['edges'][0])}")
print(f"  Scalar feature dimensions: {[f.shape for f in residue_features['node']['node_scalar_features']]}")
print(f"  Vector feature dimensions: {[f.shape for f in residue_features['node']['node_vector_features']]}")

### Feature Details

**Residue Scalar Features** include:
- One-hot encoded amino acid types (21 classes: 20 standard + UNK)
- Terminal flags (N-terminal, C-terminal)
- Intra-residue distances
- Dihedral angles (backbone φ, ψ, ω + sidechain χ angles)
- SASA (Solvent Accessible Surface Area)
- Forward/reverse connection distances

**Residue Vector Features** include:
- Intra-residue vectors
- Forward/reverse connection vectors
- Local coordinate frames

**Edge Features** include:
- Inter-residue distances (CA-CA, SC-SC, CA-SC, SC-CA)
- Relative position encoding
- Interaction vectors

## 3. Molecule Featurization

Extract features from small molecules (ligands).

In [ ]:
# Create molecule featurizer
ligand_file = '10gs_ligand.sdf'
mol_featurizer = MoleculeFeaturizer(ligand_file)

# Get features
mol_features = mol_featurizer.get_features()

print("Ligand Features:")
print(f"  Number of atoms: {mol_features['node']['node_scalar_features'][0].shape[0]}")
print(f"  Number of edges: {len(mol_features['edge']['edges'][0])}")
print(f"  Scalar feature dimensions: {[f.shape for f in mol_features['node']['node_scalar_features']]}")
print(f"  Vector feature dimensions: {[f.shape for f in mol_features['node']['node_vector_features']]}")

## 4. Working with Features

The features are returned as dictionaries with `'node'` and `'edge'` keys.
Each contains tuples of PyTorch tensors that can be used directly in graph neural networks.

In [ ]:
# Example: Access specific features
node_features = residue_features['node']
edge_features = residue_features['edge']

# Node features
residue_types_onehot = node_features['node_scalar_features'][0]  # One-hot encoded residue types
terminal_flags = node_features['node_scalar_features'][1]         # N/C-terminal flags
coordinates = node_features['coord']                               # CA and SC coordinates

# Edge features
src_nodes, dst_nodes = edge_features['edges']                      # Edge connectivity
distances = edge_features['edge_scalar_features'][0]               # Inter-residue distances
relative_positions = edge_features['edge_scalar_features'][1]      # Relative position encoding

print("\nFeature Shapes:")
print(f"  Residue types (one-hot): {residue_types_onehot.shape}")
print(f"  Terminal flags: {terminal_flags.shape}")
print(f"  Coordinates (CA + SC): {coordinates.shape}")
print(f"  Edge source nodes: {src_nodes.shape}")
print(f"  Edge distances: {distances.shape}")
print(f"  Relative positions: {relative_positions.shape}")

## 5. Handling PTMs in Featurization

When PTMs are encountered during featurization:
- PTM residues are encoded as **UNK (unknown, index 20)**
- Only **backbone atoms (N, CA, C, O) + CB** are used
- PTMs appear as **'X'** in the sequence string

This ensures consistent feature dimensions while indicating the presence of modified residues.

In [ ]:
# Check for unknown residues in the sequence
for chain_id, sequence in sequences.items():
    unk_count = sequence.count('X')
    if unk_count > 0:
        print(f"Chain {chain_id} contains {unk_count} unknown/modified residue(s) (marked as 'X')")
    else:
        print(f"Chain {chain_id} contains only standard amino acids")

## Summary

This notebook demonstrated:
1. ✓ PDB standardization with PTM handling
2. ✓ Atom-level protein featurization
3. ✓ Residue-level protein featurization with sequences
4. ✓ Molecule (ligand) featurization
5. ✓ Working with extracted features

All features are returned as PyTorch tensors ready for use in machine learning models.